In [1]:
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
X_train = pd.read_csv('./semi_project/oversampling_data/x_train_over_df.csv')
Y_train = pd.read_csv('./semi_project/oversampling_data/y_train_over_df_df.csv')
X_test = pd.read_csv('./semi_project/oversampling_data/test_data_df.csv')
Y_test = pd.read_csv('./semi_project/oversampling_data/test_label_df.csv')

In [3]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((156816, 140), (156816, 2), (36557, 140), (36557, 2))

In [4]:
Y_test['1'].value_counts()

0.0    33603
1.0     2954
Name: 1, dtype: int64

In [5]:
#체크포인트 생성및 조기종료 조건 
checkpoint_path_elu = "oversampling_deep_training/model_cp.ckpt"
checkpoint_dir_elu = os.path.dirname(checkpoint_path_elu)

elu_cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_elu, save_weights_only=False, verbose=1,)
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=500)

In [6]:
elu_neural_model = tf.keras.models.Sequential(
    [
        keras.layers.Dense(units=280, input_shape=(140,), activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0)),
        keras.layers.Dense(units=140,  activation='elu', kernel_regularizer=tf.keras.regularizers.L2(0)),
        keras.layers.Dense(units=70, activation='elu', kernel_regularizer=tf.keras.regularizers.L2(0.00)),
        keras.layers.Dense(units=40, activation='elu', kernel_regularizer=tf.keras.regularizers.L2(0.00)),
        keras.layers.Dropout(0.3),   
        keras.layers.Dense(units=1, activation='sigmoid'),
        

    ]
    ,name='predict'
)

In [7]:
elu_neural_model.summary()

Model: "predict"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 280)               39480     
                                                                 
 dense_1 (Dense)             (None, 140)               39340     
                                                                 
 dense_2 (Dense)             (None, 70)                9870      
                                                                 
 dense_3 (Dense)             (None, 40)                2840      
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 41        
                                                                 
Total params: 91,571
Trainable params: 91,571
Non-trainable

In [8]:
elu_neural_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0),
metrics=['binary_accuracy','Recall','Precision' ] )

In [9]:
history = elu_neural_model.fit(X_train, Y_train['1'].to_numpy(), epochs=2000, callbacks=[elu_cp_callback,earlystop_callback], validation_split=0.2, batch_size=30000)

Epoch 1/2000
4/5 [=======================>......] - ETA: 0s - loss: 1.3808 - binary_accuracy: 0.4996 - recall: 0.5359 - precision: 0.3817
Epoch 1: saving model to oversampling_deep_training\model_cp.ckpt
INFO:tensorflow:Assets written to: oversampling_deep_training\model_cp.ckpt\assets
5/5 [==============================] - 9s 789ms/step - loss: 1.3648 - binary_accuracy: 0.5054 - recall: 0.5142 - precision: 0.3817 - val_loss: 2.1040 - val_binary_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/2000
4/5 [=======================>......] - ETA: 0s - loss: 0.8242 - binary_accuracy: 0.5605 - recall: 0.4830 - precision: 0.4251
Epoch 2: saving model to oversampling_deep_training\model_cp.ckpt
INFO:tensorflow:Assets written to: oversampling_deep_training\model_cp.ckpt\assets
5/5 [==============================] - 1s 303ms/step - loss: 0.8186 - binary_accuracy: 0.5639 - recall: 0.4712 - precision: 0.4263 - val_loss: 1.5883 - val_binary_accuracy: 0.0000e+00 - val